In [69]:
import re
import numpy as np
import pandas as pd

In [70]:
data = pd.read_excel('Goodreads评论采集.xlsx')
content = data['评论内容']
data.head()

,书籍链接,书籍名称,语言,reviews数量,姓名,评分等级,日期,评论内容,likes数量,comments数量
0,https://www.goodreads.com/book/show/894154.Red...,Red Sorghum,English ‎,476,Mosca,5.0,"Oct 11, 2012",***************************************\r\nFor...,47.0,20.0
1,https://www.goodreads.com/book/show/894154.Red...,Red Sorghum,English ‎,476,Praj,4.0,"Apr 22, 2010",“With this book I respectfully invoke the hero...,38.0,22.0
2,https://www.goodreads.com/book/show/894154.Red...,Red Sorghum,English ‎,476,Dmitri,5.0,"Sep 04, 2021","""In late autumn, during the eighth lunar month...",34.0,10.0
3,https://www.goodreads.com/book/show/894154.Red...,Red Sorghum,English ‎,476,Ana,5.0,"Feb 14, 2014",You can say I've developed a pretty healthy ob...,34.0,11.0
4,https://www.goodreads.com/book/show/894154.Red...,Red Sorghum,English ‎,476,Alex,5.0,"Oct 17, 2017",Part of my Fall 2017 Best Of Chinese Literatur...,26.0,1.0


In [71]:
data1 = pd.read_excel('亚马逊.xlsx')
content1 = data1['review-text']
data1.head()

,标题,标题链接,头像,a-icon-alt,a-color-secondary,a-size-mini,a-section,review-text,a-color-tertiary,a-link-normal,a-text-normal,a-color-secondary1,书名
0,DFechter,https://www.amazon.com/gp/profile/amzn1.accoun...,https://images-na.ssl-images-amazon.com/images...,5.0 out of 5 stars,"Reviewed in the United States on November 11, ...",Verified Purchase,Riveting story of 1930s rural Chinese dealing ...,Imagine a character who (spoiler alert) murder...,8 people found this helpful,Report abuse,Report abuse,https://www.amazon.com/hz/reviews-render/repor...,Red Sorghum
1,K.N.R.,https://www.amazon.com/gp/profile/amzn1.accoun...,https://images-na.ssl-images-amazon.com/images...,4.0 out of 5 stars,"Reviewed in the United States on April 15, 2015",Verified Purchase,A New Classic,This is an important book. War cannot destroy ...,2 people found this helpful,Report abuse,Report abuse,https://www.amazon.com/hz/reviews-render/repor...,Red Sorghum
2,Michael D. Gilmore,https://www.amazon.com/gp/profile/amzn1.accoun...,https://images-na.ssl-images-amazon.com/images...,3.0 out of 5 stars,"Reviewed in the United States on January 26, 2016",Verified Purchase,RED SORGHUM,Perhaps because I've read all of Amy Tan and ...,One person found this helpful,Report abuse,Report abuse,https://www.amazon.com/hz/reviews-render/repor...,Red Sorghum
3,Adventure Reader,https://www.amazon.com/gp/profile/amzn1.accoun...,https://images-na.ssl-images-amazon.com/images...,4.0 out of 5 stars,"Reviewed in the United States on November 26, ...",Verified Purchase,A Very Brief Review of Red Sorghum,This is a book of 6 novella that were original...,3 people found this helpful,Report abuse,Report abuse,https://www.amazon.com/hz/reviews-render/repor...,Red Sorghum
4,BeachMama,https://www.amazon.com/gp/profile/amzn1.accoun...,https://images-na.ssl-images-amazon.com/images...,3.0 out of 5 stars,"Reviewed in the United States on July 1, 2018",Verified Purchase,Graphic and violent,This is a very graphic violent novel of Chines...,4 people found this helpful,Report abuse,Report abuse,https://www.amazon.com/hz/reviews-render/repor...,Red Sorghum


In [72]:
stop_words = []
with open('常用英文停用词(NLP处理英文必备)stopwords.txt','r',encoding='utf-8')as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append(line.strip().replace("'",""))

In [73]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word

In [74]:
def gettext(x):
#     txt = open("piao.txt","r",errors='ignore').read()
#     txt = txt.lower()
    for ch in '!"#$&()*+,-./:;<=>?@[\\]^_{|}·~‘’\n @':
        txt = str(x).replace(ch,"")
    return txt

In [75]:
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' ', tweet)
    return tweet

In [76]:
##判断是否为无效word
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [77]:
def clean_text(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', ' ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', ' ', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.lower().split()
    words = [w for w in words if w not in stop_words]
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            processed_tweet.append(word)
    return processed_tweet

In [78]:
def wenhebing(x):
    c = ' '.join(x)
    return c


In [79]:
content = content.apply(gettext)
content = content.apply(preprocess_word)
content = content.apply(handle_emojis)
content = content.apply(clean_text)
content = content.apply(wenhebing)

In [80]:
content1 = content1.apply(gettext)
content1 = content1.apply(preprocess_word)
content1 = content1.apply(handle_emojis)
content1 = content1.apply(clean_text)
content1 = content1.apply(wenhebing)

In [81]:
df = pd.DataFrame()
df['原始文本'] = data['评论内容']
df['书籍名称'] = data['书籍名称']
df['清洗文本'] = content

In [82]:
df1 = pd.DataFrame()
df1['原始文本'] = data1['review-text']
df1['书籍名称'] = data1['书名']
df1['清洗文本'] = content1

In [83]:
import paddlehub as hub
#这里使用了百度开源的成熟NLP模型来预测情感倾向
senta = hub.Module(name="senta_bilstm")
texts = df['清洗文本'].tolist()
input_data = {'text':texts}
res = senta.sentiment_classify(data=input_data)
df['情感分值'] = [x['positive_probs'] for x in res]
df.head()

[2022-02-09 22:33:31,169] [    INFO] - Installing senta_bilstm module
[2022-02-09 22:33:31,182] [    INFO] - Module senta_bilstm already installed in C:\Users\96075\.paddlehub\modules\senta_bilstm
[2022-02-09 22:33:33,565] [    INFO] - Installing lac module
[2022-02-09 22:33:33,576] [    INFO] - Module lac already installed in C:\Users\96075\.paddlehub\modules\lac


,原始文本,书籍名称,清洗文本,情感分值
0,***************************************\r\nFor...,Red Sorghum,northeastern china occupied japan brutal invas...,0.9961
1,“With this book I respectfully invoke the hero...,Red Sorghum,book respectfully invoke heroic aggrieved soul...,0.9961
2,"""In late autumn, during the eighth lunar month...",Red Sorghum,late autumn eighth lunar month vast stretches ...,0.9983
3,You can say I've developed a pretty healthy ob...,Red Sorghum,developed pretty healthy obsession mo yans wri...,0.9973
4,Part of my Fall 2017 Best Of Chinese Literatur...,Red Sorghum,fall chinese literature project here cool list...,0.9980


In [84]:
import paddlehub as hub
#这里使用了百度开源的成熟NLP模型来预测情感倾向
senta = hub.Module(name="senta_bilstm")
texts = df1['清洗文本'].tolist()
input_data = {'text':texts}
res = senta.sentiment_classify(data=input_data)
df1['情感分值'] = [x['positive_probs'] for x in res]
df1.head()

[2022-02-09 22:34:15,696] [    INFO] - Installing senta_bilstm module
[2022-02-09 22:34:15,707] [    INFO] - Module senta_bilstm already installed in C:\Users\96075\.paddlehub\modules\senta_bilstm
[2022-02-09 22:34:18,033] [    INFO] - Installing lac module
[2022-02-09 22:34:18,035] [    INFO] - Module lac already installed in C:\Users\96075\.paddlehub\modules\lac


,原始文本,书籍名称,清洗文本,情感分值
0,Imagine a character who (spoiler alert) murder...,Red Sorghum,imagine character spoiler alert murders widowe...,0.9978
1,This is an important book. War cannot destroy ...,Red Sorghum,book war destroy all family stretches generati...,0.9995
2,Perhaps because I've read all of Amy Tan and ...,Red Sorghum,read amy tan lisa novels expecting book tediou...,0.9996
3,This is a book of 6 novella that were original...,Red Sorghum,book novella originally published separately n...,0.9994
4,This is a very graphic violent novel of Chines...,Red Sorghum,graphic violent chines sex violence murder war...,0.9970


In [85]:
def pingfen(x):
    x = float(x)
    if x > 0.55:
        return 0
    elif 0.5 <= x <= 0.55:
        return 1
    else:
        return 2

In [86]:
df['情感分值'] = df['情感分值'].apply(pingfen)


In [87]:
df1['情感分值'] = df1['情感分值'].apply(pingfen)

In [88]:
df = df.replace('',value=np.NaN)
df.dropna(how='any',inplace=True)
df

,原始文本,书籍名称,清洗文本,情感分值
0,***************************************\r\nFor...,Red Sorghum,northeastern china occupied japan brutal invas...,0
1,“With this book I respectfully invoke the hero...,Red Sorghum,book respectfully invoke heroic aggrieved soul...,0
2,"""In late autumn, during the eighth lunar month...",Red Sorghum,late autumn eighth lunar month vast stretches ...,0
3,You can say I've developed a pretty healthy ob...,Red Sorghum,developed pretty healthy obsession mo yans wri...,0
4,Part of my Fall 2017 Best Of Chinese Literatur...,Red Sorghum,fall chinese literature project here cool list...,0
...,...,...,...,...
1698,I did not laugh. I should have known as the bo...,POW!,laugh book nobel prize winner award winning bo...,0
1699,"For someone who doesn’t eat meat, this book wa...",POW!,eat meat book eye opener combination filth mag...,0
1700,One of the most unique books I've ever read.,POW!,unique books read,0
1701,He suffered in comparison to my other readings...,POW!,suffered comparison readings book life short b...,0


In [89]:
df1 = df1.replace('',value=np.NaN)
df1.dropna(how='any',inplace=True)
df1

,原始文本,书籍名称,清洗文本,情感分值
0,Imagine a character who (spoiler alert) murder...,Red Sorghum,imagine character spoiler alert murders widowe...,0
1,This is an important book. War cannot destroy ...,Red Sorghum,book war destroy all family stretches generati...,0
2,Perhaps because I've read all of Amy Tan and ...,Red Sorghum,read amy tan lisa novels expecting book tediou...,0
3,This is a book of 6 novella that were original...,Red Sorghum,book novella originally published separately n...,0
4,This is a very graphic violent novel of Chines...,Red Sorghum,graphic violent chines sex violence murder war...,0
...,...,...,...,...
505,I'm afraid I have nothing intelligent to say a...,"Shifu, You'll Do Anything for a Laugh-",afraid intelligent book however reflection qua...,0
506,as expected.,"Shifu, You'll Do Anything for a Laugh-",expected,0
507,Very interesting stories.,"Shifu, You'll Do Anything for a Laugh-",stories,0
508,These short stories really come at you from a ...,"Shifu, You'll Do Anything for a Laugh-",short stories direction book literature valuab...,0


In [90]:
df.to_excel('情感分类-Goodreads评论采集.xlsx')

In [91]:
df1.to_excel('情感分类-亚马逊.xlsx')